In [1]:
#This notebook builds a heirarchial model from baseline randomforest 
#Morgan fingerprint 
#and cell painting

In [2]:
from rdkit import Chem, DataStructs
from scipy.stats import randint
from rdkit.Chem import AllChem
from scipy import stats
from rdkit.Chem import rdMolDescriptors as rdmd
from rdkit.Chem.Scaffolds import MurckoScaffold
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
from scipy.spatial.distance import cdist
from sklearn.cluster import MiniBatchKMeans
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef,confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

In [3]:
list_of_lists_df = pd.read_csv("Predictions_train_heldout_scaled_prob_all_assays_ensemble.csv")
list_of_lists_df

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled,simple_pred,Ensemble_Correct
0,178_684,InChI=1S/C46H56N4O10/c1-7-42(55)22-28-23-45(40...,0.853763,1,0.481794,0.893879,1,0.269814,1.0,Training,True,True,0.858900,0.927333,1.0,True
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.357098,0,0.481794,0.412432,1,0.269814,1.0,Training,False,True,0.370592,0.597659,0.0,False
2,178_684,InChI=1S/C31H44N4O6/c1-22-18-35(23(2)21-36)31(...,0.046336,0,0.481794,0.167905,0,0.269814,0.0,Training,True,True,0.048086,0.311150,0.0,True
3,178_684,InChI=1S/C35H60N4O5/c1-26-22-39(27(2)25-40)35(...,0.039431,0,0.481794,0.182939,0,0.269814,0.0,Training,True,True,0.040921,0.339009,0.0,True
4,178_684,InChI=1S/C29H39N5O5/c1-19-16-34(20(2)18-35)28(...,0.345300,0,0.481794,0.091189,0,0.269814,0.0,Training,True,True,0.358348,0.168984,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58482,93_259,InChI=1S/C12H11N3O2S/c1-8-7-9(2)15(13-8)12-10-...,0.127744,0,0.245120,0.513325,1,0.428485,0.0,HeldOut,True,False,0.260575,0.574224,0.0,True
58483,93_259,InChI=1S/C17H16N2O4S/c1-22-14-7-8-16(23-2)17(1...,0.505973,1,0.245120,0.322499,0,0.428485,0.0,HeldOut,False,True,0.672778,0.376325,1.0,False
58484,93_259,InChI=1S/C24H21N3O2S/c28-23-19-8-4-5-9-21(19)3...,0.363862,1,0.245120,0.359675,0,0.428485,1.0,HeldOut,True,False,0.578650,0.419705,0.0,False
58485,93_259,InChI=1S/C18H22N2O4S/c1-14(2)15-5-7-16(8-6-15)...,0.290175,1,0.245120,0.260526,0,0.428485,0.0,HeldOut,False,True,0.529843,0.304008,0.0,True


In [4]:
from sklearn.linear_model import LogisticRegression
from io import StringIO
from itertools import product
import sys
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold

In [5]:
def RF_Fn(assay):

    detail_list=pd.DataFrame()
    detail_list = list_of_lists_df[list_of_lists_df["assay"]==assay].reset_index(drop=True)
    
    
    proba_batch=[]
    pred_batch=[]
    true_batch=[]
    StdInChI_batch=[]
    
    train= detail_list[detail_list["Data"]=="Training"]

    test= detail_list[detail_list["Data"]=="HeldOut"]
    

    print("Herirachal model for training data")    

    print("RF Model")
            
    X_train = train[["fp_proba_scaled", "CP_proba_scaled"]].to_numpy()
    y_train = train["true"].to_numpy()
    X_test = test[["fp_proba_scaled", "CP_proba_scaled"]].to_numpy()
    y_test = test["true"].to_numpy()


    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
         
    param_dist_grid = {
        'n_estimators':[100, 300, 400, 500],
        'random_state': [42],
        'n_jobs': [1],
        'class_weight' : [None, 'balanced']
        }
        
    rf = RandomForestClassifier(n_jobs=-1)
    rsh = HalvingRandomSearchCV(estimator=rf, param_distributions=param_dist_grid, factor=2, 
                                random_state=42, n_jobs=20, verbose=0,  cv = inner_cv)

    rsh.fit(X_train, y_train)

    print("Tuned Mode: ", rsh.best_params_)
    clf = rsh.best_estimator_
        
    #n_estimators=100
    #clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=10, random_state=42, class_weight="balanced")           
    clf.fit(X_train, y_train)
    
    #Threshold Balancing
    cross_val_prob = cross_val_predict(rsh.best_estimator_, X_train, y_train, cv=inner_cv, method='predict_proba')[:, 1]
    # calculate roc curves
    fpr, tpr, thresholds = roc_curve(y_train, cross_val_prob)
    # get the best threshold
    J = tpr - fpr
    ix = argmax(J)
    best_thresh = thresholds[ix]
    print('Best Threshold=%f' % (best_thresh))


    proba = clf.predict_proba(X_test)[:,1]
    pred  = [ 1 if x>best_thresh  else 0 for x in proba ] 

           
    StdInChI = test["InChICode_standardised"].to_numpy()
    #pred = clf.predict(X_test) 
    #proba = clf.predict_proba(X_test)[:,1]
    true = test["true"].to_numpy()
            

    StdInChI_batch = np.concatenate([StdInChI_batch, StdInChI])
    proba_batch = np.concatenate([proba_batch, proba])
    pred_batch = np.concatenate([pred_batch, pred])
    true_batch = np.concatenate([true_batch, true])

    print(len(pred_batch))
    print(len(true_batch))
   
    return StdInChI_batch, true_batch, pred_batch, proba_batch

In [6]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score

def check_heirarchial(assay, method, y_true, y_pred, y_prob):
    
    conf_matrix = confusion_matrix(y_true, y_pred)
    #print(conf_matrix)
    
    #print(classification_report(y_true, y_pred))
    
    ba= balanced_accuracy_score(y_true, y_pred)
    #print('balanced_accuracy_score ',ba)
    
    mcc=matthews_corrcoef(y_true, y_pred)
    #print('matthews_corrcoef ',mcc)
    # Sensitivity, hit rate, recall, or true positive rate
    
    f1= f1_score(y_true, y_pred, average='binary')
    precision = precision_score(y_true, y_pred, average='binary')
    
    #print('F1 Toxic', f1)
    #print('precision', precision)
    
    Specificity = conf_matrix[0,0]/(conf_matrix[0,0]+conf_matrix[0,1])
    # Specificity or true negative rate
    Sensitivity = conf_matrix[1,1]/(conf_matrix[1,0]+conf_matrix[1,1]) 
    
    #print( 'Sensitivity', Sensitivity)
    #print( 'Specificity', Specificity)
    AUC = roc_auc_score(y_true, y_prob)
    #print('AUC-ROC ',AUC)    
    # calculate roc curves
    best_model_fpr, best_model_tpr, _ = roc_curve(y_true, y_prob)   
    
    
    row=[assay, method, f1, precision, Sensitivity, Specificity, ba, mcc, AUC]
    metrics_list.append(row)
    
    return metrics_list

In [7]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict
from collections import Counter
from numpy import argmax
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

In [8]:
import warnings
warnings.filterwarnings("ignore") #ignore, default
import pandas

#sns.set(rc={'figure.figsize':(2, 2)})
#plt.rcParams['figure.dpi'] = 80
#sns.set_style("white")
metrics_list=[]
all_preds_test_compounds=pd.DataFrame(columns=['assay', 'InChICode_standardised', 'fp_proba', 'fp_pred', 'fp_threshold', 'CP_proba',
       'CP_pred', 'CP_threshold', 'true', 'Data', 'MFP_Correct',
       'CP_Correct', 'fp_proba_scaled', 'CP_proba_scaled', 'simple_pred',
       'Ensemble_Correct', 'heirarchial_pred', 'heirarchial_Correct'])


#for assay in tqdm(assaylist[:7]):
for assay in tqdm(list_of_lists_df.assay.unique()[:]):
    
    print("Assay: ", assay)
    detail_list=pd.DataFrame()
    detail_list = list_of_lists_df[list_of_lists_df["assay"]==assay].reset_index(drop=True)
    detail_list_test = detail_list[detail_list["Data"]=="HeldOut"]

    print("Heirarchial_Model")
    merger_df=pd.DataFrame()
    
    StdInChI_batch, true_batch, pred_batch, proba_batch = RF_Fn(assay)
    check_heirarchial(assay, "Heirarchial Model", true_batch, pred_batch, proba_batch)
    
    #print(true_batch)
    
    print(assay)
    
    merger_df["InChICode_standardised"]=StdInChI_batch
    merger_df["heirarchial_pred"]= pred_batch
    merger_df["true"]=true_batch
    merger_df["heirarchial_Correct"]= merger_df["heirarchial_pred"] == merger_df["true"]
    merger_df["assay"]=assay
    
    
    #For each assay combine original predictions and merger_df predictions
    #all_preds = pd.DataFrame()
    all_preds_test_compounds= pd.concat([all_preds_test_compounds, pd.merge(detail_list_test, merger_df)])  
    
    
    

  0%|                                                    | 0/89 [00:00<?, ?it/s]

Assay:  178_684
Heirarchial_Model
Herirachal model for training data
RF Model
Tuned Mode:  {'random_state': 42, 'n_jobs': 1, 'n_estimators': 500, 'class_weight': 'balanced'}


  1%|▍                                           | 1/89 [00:06<09:26,  6.43s/it]

Best Threshold=0.436000
55
55
178_684
Assay:  180_686
Heirarchial_Model
Herirachal model for training data
RF Model
Tuned Mode:  {'random_state': 42, 'n_jobs': 1, 'n_estimators': 100, 'class_weight': 'balanced'}


  2%|▉                                           | 2/89 [00:08<05:46,  3.98s/it]

...

Best Threshold=0.134000
108
108
280_759
Assay:  283_762
Heirarchial_Model
Herirachal model for training data
RF Model
Tuned Mode:  {'random_state': 42, 'n_jobs': 1, 'n_estimators': 500, 'class_weight': 'balanced'}


 99%|██████████████████████████████████████████▌| 88/89 [07:19<00:05,  5.33s/it]

Best Threshold=0.206000
86
86
283_762
Assay:  93_259
Heirarchial_Model
Herirachal model for training data
RF Model
Tuned Mode:  {'random_state': 42, 'n_jobs': 1, 'n_estimators': 400, 'class_weight': None}


100%|███████████████████████████████████████████| 89/89 [07:23<00:00,  4.98s/it]

Best Threshold=0.570000
26
26
93_259


In [9]:
all_preds_test_compounds

,assay,InChICode_standardised,fp_proba,fp_pred,fp_threshold,CP_proba,CP_pred,CP_threshold,true,Data,MFP_Correct,CP_Correct,fp_proba_scaled,CP_proba_scaled,simple_pred,Ensemble_Correct,heirarchial_pred,heirarchial_Correct
0,178_684,InChI=1S/C24H28N4O4S/c1-14-12-28(15(2)13-29)24...,0.431564,1,0.420981,0.687784,1,0.306156,1.0,HeldOut,True,True,0.509139,0.775010,1.0,True,1.0,True
1,178_684,InChI=1S/C30H42ClN3O5S/c1-21-17-34(22(2)20-35)...,0.431601,1,0.420981,0.186083,0,0.306156,0.0,HeldOut,False,True,0.509171,0.303902,0.0,True,0.0,True
2,178_684,InChI=1S/C25H35N3O5S/c1-17-10-12-20(13-11-17)3...,0.105376,0,0.420981,0.074175,0,0.306156,0.0,HeldOut,True,True,0.125156,0.121139,0.0,True,0.0,True
3,178_684,InChI=1S/C31H42N4O6/c1-20-15-35(21(2)18-36)30(...,0.173670,0,0.420981,0.061583,0,0.306156,0.0,HeldOut,True,True,0.206269,0.100575,0.0,True,0.0,True
4,178_684,InChI=1S/C30H41N5O5/c1-20-17-35(21(2)19-36)28(...,0.307765,0,0.420981,0.083022,0,0.306156,1.0,HeldOut,False,False,0.365534,0.135587,0.0,False,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,93_259,InChI=1S/C12H11N3O2S/c1-8-7-9(2)15(13-8)12-10-...,0.127744,0,0.245120,0.513325,1,0.428485,0.0,HeldOut,True,False,0.260575,0.574224,0.0,True,0.0,True
22,93_259,InChI=1S/C17H16N2O4S/c1-22-14-7-8-16(23-2)17(1...,0.505973,1,0.245120,0.322499,0,0.428485,0.0,HeldOut,False,True,0.672778,0.376325,1.0,False,0.0,True
23,93_259,InChI=1S/C24H21N3O2S/c28-23-19-8-4-5-9-21(19)3...,0.363862,1,0.245120,0.359675,0,0.428485,1.0,HeldOut,True,False,0.578650,0.419705,0.0,False,0.0,False
24,93_259,InChI=1S/C18H22N2O4S/c1-14(2)15-5-7-16(8-6-15)...,0.290175,1,0.245120,0.260526,0,0.428485,0.0,HeldOut,False,True,0.529843,0.304008,0.0,True,0.0,True


In [10]:
all_preds_test_compounds.to_csv("Predictions_train_heldout_scaled_prob_all_assays_heirarchial_model_test_compounds.csv", index=False)

In [11]:
metrics_list = pd.DataFrame(metrics_list,columns=["assay","method","f1", "precision", "Sensitivity", "Specificity", "ba", "mcc", "AUC"])
metrics_list

,assay,method,f1,precision,Sensitivity,Specificity,ba,mcc,AUC
0,178_684,Heirarchial Model,0.583333,0.700000,0.500000,0.926829,0.713415,0.482063,0.770035
1,180_686,Heirarchial Model,0.370370,0.294118,0.500000,0.612903,0.556452,0.098414,0.558065
2,183_604,Heirarchial Model,0.500000,0.571429,0.444444,0.888889,0.666667,0.364698,0.860082
3,184_606,Heirarchial Model,0.848485,0.823529,0.875000,0.936170,0.905585,0.795459,0.953457
4,185_665,Heirarchial Model,0.375000,0.300000,0.500000,0.611111,0.555556,0.097590,0.587963
...,...,...,...,...,...,...,...,...,...
84,278_757,Heirarchial Model,0.502924,0.447917,0.573333,0.766520,0.669927,0.315330,0.708488
85,279_758,Heirarchial Model,0.551724,0.500000,0.615385,0.800000,0.707692,0.389306,0.742308
86,280_759,Heirarchial Model,0.603175,0.527778,0.703704,0.790123,0.746914,0.453609,0.764746
87,283_762,Heirarchial Model,0.294118,0.212766,0.476190,0.430769,0.453480,-0.080289,0.463736


In [12]:
metrics_list.to_csv("heirarchial_model_metrics.csv", index=False)